In [ ]:
#在/Data/Ref下创建nam亲本各自的目录，完善内容
#!/bin/bash
cd /data14/yjiang22/qukunlab_atac_pipe/ATAC-pipe-master/Data/Ref
source activate atac-pipe
##cml52
mkdir cml52 && cd cml52
wget https://download.maizegdb.org/Zm-CML52-REFERENCE-NAM-1.0/Zm-CML52-REFERENCE-NAM-1.0.fa.gz
wget https://download.maizegdb.org/Zm-CML52-REFERENCE-NAM-1.0/Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.gff3.gz
gunzip -c Zm-CML52-REFERENCE-NAM-1.0.fa.gz > Zm-CML52-REFERENCE-NAM-1.0.fa
gunzip Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.gff3.gz
#创建基因组黑名单
touch cml52.blacklist.bed
#创建6个bt2文件
for i in {1..10}
do
    samtools faidx Zm-CML52-REFERENCE-NAM-1.0.fa chr$i > chr$i.fa
done
cat chr1.fa chr2.fa chr3.fa chr4.fa chr5.fa chr6.fa chr7.fa chr8.fa chr9.fa chr10.fa > cml52.fa
bowtie2-build cml52.fa cml52
#创建sizes文件
samtools faidx Zm-CML52-REFERENCE-NAM-1.0.fa
cat *fai | head -10 | cut -f 1,2 > cml52.sizes
#创建NAME_refseq_genes_TSS.txt (gfftobed是github上down的一个包, -t指提取TSS)
/data14/yjiang22/atac_pipe1/genome/gfftobed-main/gfftobed -t Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.gff3 | cut -f 1,2,3 | grep chr > cml52_refseq_genes_TSS.txt
#创建4kbwindow_NAMETSS.txt
cat cml52_refseq_genes_TSS.txt | awk 'BEGIN{FS="\t";OFS="\t"}{print $1,$2-2000,$3+2000}' > 4kbwindow_cml52TSS.txt

#为了减少占用空间，删除一些不必保留的文件
rm chr*.fa
-----------------------------------------------------------------------------------------------------------------------------------------
#!/bin/bash
#b97
cd /data14/yjiang22/qukunlab_atac_pipe/ATAC-pipe-master/Data/Ref
source activate atac-pipe
mkdir b97 && cd b97
wget https://download.maizegdb.org/Zm-B97-REFERENCE-NAM-1.0/Zm-B97-REFERENCE-NAM-1.0.fa.gz
wget https://download.maizegdb.org/Zm-B97-REFERENCE-NAM-1.0/Zm-B97-REFERENCE-NAM-1.0_Zm00018ab.1.gff3.gz
gunzip -c Zm-B97-REFERENCE-NAM-1.0.fa.gz > Zm-B97-REFERENCE-NAM-1.0.fa
gunzip Zm-B97-REFERENCE-NAM-1.0_Zm00018ab.1.gff3.gz
#创建基因组黑名单
touch b97.blacklist.bed
#创建6个bt2文件
for i in {1..10}
do
    samtools faidx Zm-B97-REFERENCE-NAM-1.0.fa chr$i > chr$i.fa
done
cat chr1.fa chr2.fa chr3.fa chr4.fa chr5.fa chr6.fa chr7.fa chr8.fa chr9.fa chr10.fa > b97.fa
bowtie2-build b97.fa b97
#创建sizes文件
samtools faidx Zm-B97-REFERENCE-NAM-1.0.fa
cat *fai | head -10 | cut -f 1,2 > b97.sizes
#创建NAME_refseq_genes_TSS.txt (gfftobed是github上down的一个包, -t指提取TSS)
/data14/yjiang22/atac_pipe1/genome/gfftobed-main/gfftobed -t Zm-B97-REFERENCE-NAM-1.0_Zm00018ab.1.gff3 | cut -f 1,2,3 | grep chr > b97_refseq_genes_TSS.txt
#创建4kbwindow_NAMETSS.txt
cat b97_refseq_genes_TSS.txt | awk 'BEGIN{FS="\t";OFS="\t"}{print $1,$2-2000,$3+2000}' > 4kbwindow_b97TSS.txt

#为了减少占用空间，删除一些不必保留的文件
rm Zm-B97-REFERENCE-NAM-1.0.fa
rm Zm-B97-REFERENCE-NAM-1.0.fa.fai
rm chr*.fa

-----------------------------------------------------------------------------------------------------------------------------------------
##-----------------------------------------------------------循环 构造Data/Ref---------------------------------------------------------------------##
#!/bin/bash
source activate atac-pipe
cd /data14/yjiang22/qukunlab_atac_pipe/ATAC-pipe-master/Data/Ref
cat ../id_genome_gff_http | while read id
do
arr=($id)
id_lower=${arr[0]}
id_upper=${arr[1]}
genome_http=${arr[2]}
gff_http=${arr[3]}
#创建各基因组目录并进入
mkdir $id_lower && cd $id_lower
#下载基因组文件和注释文件
wget $genome_http
wget $gff_http
gunzip -c Zm-${id_upper}-REFERENCE-NAM-1.0.fa.gz > Zm-${id_upper}-REFERENCE-NAM-1.0.fa
gunzip *.gff3.gz
#创建基因组黑名单
touch $id_lower.blacklist.bed
#创建6个bt2文件
for i in {1..10}
do
    samtools faidx Zm-${id_upper}-REFERENCE-NAM-1.0.fa chr$i > chr$i.fa
done
cat chr1.fa chr2.fa chr3.fa chr4.fa chr5.fa chr6.fa chr7.fa chr8.fa chr9.fa chr10.fa > ${id_lower}.fa
bowtie2-build $id_lower.fa $id_lower
#创建sizes文件
samtools faidx Zm-$id_upper-REFERENCE-NAM-1.0.fa
cat *fai | head -10 | cut -f 1,2 > $id_lower.sizes
#创建NAME_refseq_genes_TSS.txt (gfftobed是github上down的一个包, -t指提取TSS)
/data14/yjiang22/atac_pipe1/genome/gfftobed-main/gfftobed -t *.gff3 | cut -f 1,2,3 | grep chr > ${id_lower}_refseq_genes_TSS.txt
#创建4kbwindow_NAMETSS.txt
cat *TSS.txt | awk 'BEGIN{FS="\t";OFS="\t"}{print $1,$2-2000,$3+2000}' > 4kbwindow_${id_lower}TSS.txt

#为了减少占用空间，删除一些不必保留的文件
rm Zm-${id_upper}-REFERENCE-NAM-1.0.fa
rm Zm-${id_upper}-REFERENCE-NAM-1.0.fa.fai
rm chr*.fa

cd ..
done

##-----------------------------------------------------------循环 构造Data/Ref---------------------------------------------------------------------##


#cml247
wget https://download.maizegdb.org/Zm-CML247-REFERENCE-NAM-1.0/Zm-CML247-REFERENCE-NAM-1.0.fa.gz
wget https://download.maizegdb.org/Zm-CML247-REFERENCE-NAM-1.0/Zm-CML247-REFERENCE-NAM-1.0_Zm00023ab.1.gff3.gz

...


### 提交并行作业

#### SRA转fastq

In [ ]:
###-----------------SRA转fastq-----------------###


#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 90
#SBATCH -o /public1/home/sc80041/yjiang22/nam/sra2fastq.log

source activate jy_atac
cd /public1/home/sc80041/yjiang22/nam
cat ./sra2fastq--.txt | while read id
do
    arr=($id)
    sra_id=${arr[0]}
    fastq_id_base=${arr[1]}
    #echo $sra_id
    /public1/home/sc80041/yjiang22/software/sratoolkit.3.0.0-centos_linux64/bin/fasterq-dump -e 84 --split-3 -O ./fastq/ ./raw_data/$sra_id

done


#### fastq文件批量更名

In [ ]:
###-----------------fastq更名-----------------###
cat ../config | while read id;do arr=($id);sra_id=${arr[0]};fastq_id_base=${arr[1]};rename ${sra_id} ${fastq_id_base} *;done
rename fastq fq *
再新建26个文件夹把各自的fq放到相应的文件夹里


#### 修改atac-pipe_NAM_mt.py

In [ ]:
###----------------修改atac-pipe_NAM_mt.py------------------###
(base) [yjiang22@localhost Ref]$ ls | while read id;do echo $id;cat ./$id/*.sizes | awk '{sum+=$2} END {print sum}';done


#### 寻找nam中的pt mt

In [ ]:
###--------------------寻找nam中的pt mt----------------------###
#策略：将b73 v4的mt pt分别比对到26个nam群体的基因组

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 50
#SBATCH -o /public1/home/sc80041/yjiang22/nam/find_nam_mt_pt.log
source activate minimap2
cd /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref
ls | while read id;do minimap2 -ax asm5 -t 20 $id/*NAM*.fa /public1/home/sc80041/yjiang22/nam/b73v4_mt_pt.fa | samtools view -@ 40 -b -o /public1/home/sc80041/yjiang22/nam/find_nam_mt_pt/${id}_where_mt_pt.bam;done
cd /public1/home/sc80041/yjiang22/nam/find_nam_mt_pt
cat ../nam26_lowercase | while read id;do perl ../get_the_detailed_mapping_information_from_bwa_bam_file.pl ${id}_where_mt_pt.bam 0 > ${id}_where_mt_pt.bam.info;done

#总结：由于b73 v4的叶绿体、线粒体比对到多个nam亲本的常染色体上,分离所有nam亲本的叶绿体线粒体较复杂,暂不进行统计


#### qukunlab_atac_pipe mapping QC

In [ ]:
###--------------------qukunlab_atac_pipe mapping QC----------------------###

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 50
#SBATCH -o /public1/home/sc80041/yjiang22/nam/log/b97mappQC.log
source activate jy_atac
python atac-pipe_NAM_mt.py --MappingQC -t 40 -i /public1/home/sc80041/yjiang22/nam/fastq/b97/ -o /public1/home/sc80041/yjiang22/nam/out/ -r b97

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 42
#SBATCH -o /public1/home/sc80041/yjiang22/nam/log/b73v5mappQC_1.log
source activate jy_atac
python atac-pipe_NAM_mt.py --MappingQC -t 40 -i /public1/home/sc80041/yjiang22/nam/fastq/b73v5/fastq1/ -o /public1/home/sc80041/yjiang22/nam/out/ -r b73v5

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/log/b73v5mappQC_2.log
source activate jy_atac
python atac-pipe_NAM_mt.py --MappingQC -t 94 -i /public1/home/sc80041/yjiang22/nam/fastq/b73v5/fastq2/ -o /public1/home/sc80041/yjiang22/nam/out/ -r b73v5

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/log/b73v5mappQC_3.log
source activate jy_atac
python atac-pipe_NAM_mt.py --MappingQC -t 94 -i /public1/home/sc80041/yjiang22/nam/fastq/b73v5/fastq3/ -o /public1/home/sc80041/yjiang22/nam/out/ -r b73v5


#######为了节约存储空间，该步完成后把fastq文件gzip压缩
#######b73分了三个作业，因为fastq文件过大，跑不动

### qukunlab_atac_pipe merge

In [ ]:
###--------------------qukunlab_atac_pipe merge----------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/log/nam_merge.log
source activate jy_atac

cat /public1/home/sc80041/yjiang22/nam/nam26_lowercase | while read id   #change b73v5 to b73
do
    python atac-pipe_NAM_mt.py \
    --Merge \
    --bam /public1/home/sc80041/yjiang22/nam/out/Mapping/$id/ \
    -r $id \
    -o /public1/home/sc80041/yjiang22/nam/out/ \
    --group /public1/home/sc80041/yjiang22/nam/group/${id}_group
done



#### qukunlab_atac_pipe peakcalling

In [ ]:

###--------------------qukunlab_atac_pipe peak calling----------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 90
#SBATCH -o /public1/home/sc80041/yjiang22/nam/log/nam_peakcall.log
source activate jy_atac
cat /public1/home/sc80041/yjiang22/nam/nam26_lowercase | while read id   #change b73v5 to b73
do
    python atac-pipe_NAM_mt.py \
    --PeakCalling \
    --bed /public1/home/sc80041/yjiang22/nam/out/Mapping/$id/ \
    -r $id \
    -o /public1/home/sc80041/yjiang22/nam/out/ \
    --group /public1/home/sc80041/yjiang22/nam/group/${id}_group \
    --project $id
done

#### bedtools getfasta提取peak序列

发现：多个某chr上最后一个peak，bed文件坐标超出了染色体坐标，getfasta时报错，需要手动修改！！！！！！

In [ ]:
##准备peak坐标的bed文件
ls | cut -d "_" -f 1|uniq| while read id;do cut -f 1,2,3 ${id}_peaks.narrowPeak > ${id}_peaks.bed;done
##将peaks.bed文件超出染色体长度的坐标改为染色体长度
如
Feature (chr10:150885159-150885482) beyond the length of chr10 size (150885398 bp).  Skipping.
##提取peak序列
ls | cut -d '_' -f 1 |  while read id;do bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${id}/*-NAM*.fa -bed ${id}_peaks.bed -fo ${id}_peaks.woparent.fa;done
##给每个fa文件header添加亲本名称信息
#单个
sed 's/>/&b73/' b73_peaks.woparent.fa > b73_peaks.fa
#批量
ls | cut -d "_" -f 1|uniq| while read id;do echo "sed 's/>/&$id/' ${id}_peaks.woparent.fa > ${id}_peaks.fa" >> sed_add_fa_header.sh ;done #再运行.sh，sed 's/>/&$id/直接执行把$id当成'$id'了



#### minimap2比对，迭代构建pan-ocr(只考虑unmapped)

In [ ]:
b73 b97 cml52 cml69 cml103 cml228 cml247 cml277 cml322 cml333 hp301 il14h ki3 ki11 ky21 m37w m162w mo18w ms71 nc350 nc358 oh7b oh43 p39 tx303 tzi8

##第1次，b97(query)比对到b73(ref)

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/minim_b97_b73ref_aln.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/minimap2_trial1
minimap2 -t 28 -a b73--ref.fa ../fasta_peaks/b97_peaks.fa | samtools view -b -o b97.minim.bam
#samtools sort -@ 28 -n b97.minim.bam -o b97.minim.sortbyread.bam  #-n sort by read name
#perl /public1/home/sc80041/yjiang22/nam/get_the_detailed_mapping_information_from_bwa_bam_file.pl b97-b73.minim.sortbyread.bam 0 > b97-b73.minim.sortbyread.bam.info  ###query name那块被修改了
#samtools view b97-b73.minim.sortbyread.bam |awk '$2==4 {print $1}' > b97-b73.minim.unmapped
samtools view b97.minim.bam | awk '$2==4 {print $1}' > b97.minim.unmapped
#构造用来提取unmapped reads序列的bed文件
awk -F ":|-" '{print $1,$2,$3}' OFS="\t" b97.minim.unmapped > b97.minim.unmapped.bedinterm
sed 's/b97//' b97.minim.unmapped.bedinterm > b97.minim.unmapped.bed
#提取unmapped reads序列 给fa文件header加亲本信息
bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b97/Zm-B97-REFERENCE-NAM-1.0.fa -bed b97.minim.unmapped.bed -fo b97.minim.unmapped.woparent.fa
sed 's/>/&b97/' b97.minim.unmapped.woparent.fa > b97.minim.unmapped.fa
cat b97.minim.unmapped.fa b73--ref.fa > b97--ref.fa


#########-----------循环minimap2-iterate.sh------------##########
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/minim_iterate_26nam.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/minimap2_trial1
cat ../iterate-config-ref-query--.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    minimap2 -t 28 -a ${ref}--ref.fa ../fasta_peaks/${query}_peaks.fa | samtools view -b -o ${query}.minim.bam
    samtools view ${query}.minim.bam | awk '$2==4 {print $1}' > ${query}.minim.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ${query}.minim.unmapped > ${query}.minim.unmapped.bedinterm
    bash ./sed_bedinterm2bed.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM*.fa -bed ${query}.minim.unmapped.bed -fo ${query}.minim.unmapped.woparent.fa
    bash ./sed_add_fa_parent.sh
    cat ${query}.minim.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
#########-----------循环minimap2-iterate.sh------------##########


#########-----------循环minimap2-iterate2.sh------------##########
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/minim_iterate_26nam_2.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/minimap2_trial2
cat ../iterate-config-ref-query--2.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    minimap2 -t 28 -a ${ref}--ref.fa ../fasta_peaks/${query}_peaks.fa | samtools view -b -o ${query}.minim.bam
    samtools view ${query}.minim.bam | awk '$2==4 {print $1}' > ${query}.minim.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ${query}.minim.unmapped > ${query}.minim.unmapped.bedinterm
    bash ./sed_bedinterm2bed2.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM*.fa -bed ${query}.minim.unmapped.bed -fo ${query}.minim.unmapped.woparent.fa
    bash ./sed_add_fa_parent2.sh
    cat ${query}.minim.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
#########-----------循环minimap2-iterate2.sh------------##########

#########-----------循环minimap2-iterate3.sh------------##########
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/minim_iterate_26nam_3.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/minimap2_trial3
cat ../iterate-config-ref-query--3.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    minimap2 -t 28 -a ${ref}--ref.fa ../fasta_peaks/${query}_peaks.fa | samtools view -b -o ${query}.minim.bam
    samtools view ${query}.minim.bam | awk '$2==4 {print $1}' > ${query}.minim.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ${query}.minim.unmapped > ${query}.minim.unmapped.bedinterm
    bash ./sed_bedinterm2bed3.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM*.fa -bed ${query}.minim.unmapped.bed -fo ${query}.minim.unmapped.woparent.fa
    bash ./sed_add_fa_parent3.sh
    cat ${query}.minim.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
#########-----------循环minimap2-iterate3.sh------------##########

105567 b73_peaks.bed
21980 oh43.minim.unmapped    127547
5931 ky21.minim.unmapped     133478
4152 ki3.minim.unmapped      137630
3259 m162w.minim.unmapped    140889
2526 nc350.minim.unmapped    143415
2094 oh7b.minim.unmapped     145509
2425 b97.minim.unmapped      147934
2110 il14h.minim.unmapped    150044
1840 tx303.minim.unmapped    151884
1547 m37w.minim.unmapped     153431
1549 hp301.minim.unmapped    154980
1452 cml69.minim.unmapped    156432
1399 cml277.minim.unmapped   157831
1442 ms71.minim.unmapped     159273
1518 cml333.minim.unmapped   160791
1456 cml247.minim.unmapped   162247
1425 p39.minim.unmapped      163672
1388 tzi8.minim.unmapped     165060
1176 nc358.minim.unmapped    166236
1404 cml52.minim.unmapped    167640
958 cml322.minim.unmapped    168598
883 ki11.minim.unmapped      169481
1155 cml103.minim.unmapped   170636
985 mo18w.minim.unmapped     171621
1366 cml228.minim.unmapped   172987

#panocr共172987条

#### blast+比对，迭代构建pan-ocr(只考虑Unmapped)

In [ ]:
#先创建yjiang22环境，conda安装ncbi-blast+
source activate yjiang22
makeblastdb -in b73--ref.fa -dbtype nucl -out b73--ref -parse_seqids
blastn -db b73--ref -query ../../fasta_peaks/b97_peaks.fa -outfmt 6 -out ../b97.blast -num_threads 20
cut -f 1 ../b97.blast | sort  | uniq > ../b97.blast.mapped
#总-mapped得到umapped  a.txt-b.txt差集  sort a.txt b.txt b.txt | uniq -u
grep '>' ../../fasta_peaks/b97_peaks.fa | sed 's/>//' > ../b97_peaks_all
sort ../b97_peaks_all ../b97.blast.mapped ../b97.blast.mapped | uniq -u > ../b97.blast.unmapped
#构造用来提取unmapped reads序列的bed文件
awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../b97.blast.unmapped > ../b97.blast.unmapped.bedinterm
sed 's/b97//' ../b97.blast.unmapped.bedinterm > ../b97.blast.unmapped.bed
#提取unmapped reads序列 给fa文件header加亲本信息
bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b97/Zm-B97-REFERENCE-NAM-1.0.fa -bed ../b97.blast.unmapped.bed -fo b97.blast.unmapped.woparent.fa
sed 's/>/&b97/' b97.blast.unmapped.woparent.fa > b97.blast.unmapped.fa
#加到ref的fa里面继续下一轮比对
cat b97.blast.unmapped.fa b73--ref.fa > b97--ref.fa


###---------------------------循环 blast_iterate.sh------------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 22
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/blast_iterate_26nam_1.log
source activate yjiang22
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/blast_trial1/fa_and_index1
cat ../../iterate-config-ref-query--.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    makeblastdb -in ${ref}--ref.fa -dbtype nucl -out ${ref}--ref -parse_seqids
    blastn -db ${ref}--ref -query ../../fasta_peaks/${query}_peaks.fa -outfmt 6 -out ../${query}.blast -num_threads 20
    cut -f 1 ../${query}.blast | sort  | uniq > ../${query}.blast.mapped
    grep '>' ../../fasta_peaks/${query}_peaks.fa | sed 's/>//' > ../${query}_peaks_all
    sort ../${query}_peaks_all ../${query}.blast.mapped ../${query}.blast.mapped | uniq -u > ../${query}.blast.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../${query}.blast.unmapped > ../${query}.blast.unmapped.bedinterm
    bash ./sed_bedinterm2bed.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM-*.fa -bed ../${query}.blast.unmapped.bed -fo ${query}.blast.unmapped.woparent.fa
    bash ./sed_add_fa_parent.sh
    cat ${query}.blast.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
###---------------------------循环 blast_iterate.sh------------------------###

#panocr共166543条

###---------------------------循环 blast_iterate2.sh------------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 22
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/blast_iterate_26nam_2.log
source activate yjiang22
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/blast_trial2/fa_and_index2
cat ../../iterate-config-ref-query--2.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    makeblastdb -in ${ref}--ref.fa -dbtype nucl -out ${ref}--ref -parse_seqids
    blastn -db ${ref}--ref -query ../../fasta_peaks/${query}_peaks.fa -outfmt 6 -out ../${query}.blast -num_threads 20
    cut -f 1 ../${query}.blast | sort  | uniq > ../${query}.blast.mapped
    grep '>' ../../fasta_peaks/${query}_peaks.fa | sed 's/>//' > ../${query}_peaks_all
    sort ../${query}_peaks_all ../${query}.blast.mapped ../${query}.blast.mapped | uniq -u > ../${query}.blast.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../${query}.blast.unmapped > ../${query}.blast.unmapped.bedinterm
    bash ./sed_bedinterm2bed_2.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM-*.fa -bed ../${query}.blast.unmapped.bed -fo ${query}.blast.unmapped.woparent.fa
    bash ./sed_add_fa_parent_2.sh
    cat ${query}.blast.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
###---------------------------循环 blast_iterate2.sh------------------------###



###---------------------------循环 blast_iterate3.sh------------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 22
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/blast_iterate_26nam_3.log
source activate yjiang22
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/blast_trial3/fa_and_index3
cat ../../iterate-config-ref-query--3.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    makeblastdb -in ${ref}--ref.fa -dbtype nucl -out ${ref}--ref -parse_seqids
    blastn -db ${ref}--ref -query ../../fasta_peaks/${query}_peaks.fa -outfmt 6 -out ../${query}.blast -num_threads 20
    cut -f 1 ../${query}.blast | sort  | uniq > ../${query}.blast.mapped
    grep '>' ../../fasta_peaks/${query}_peaks.fa | sed 's/>//' > ../${query}_peaks_all
    sort ../${query}_peaks_all ../${query}.blast.mapped ../${query}.blast.mapped | uniq -u > ../${query}.blast.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../${query}.blast.unmapped > ../${query}.blast.unmapped.bedinterm
    bash ./sed_bedinterm2bed_3.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM-*.fa -bed ../${query}.blast.unmapped.bed -fo ${query}.blast.unmapped.woparent.fa
    bash ./sed_add_fa_parent_3.sh
    cat ${query}.blast.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
###---------------------------循环 blast_iterate3.sh------------------------###


#### bwa-mem比对，迭代构建pan-ocr(只考虑unmapped)

In [ ]:

source activate bwa
cd
#建立索引
bwa index b73_peaks.fa -p b73--ref
#bwa mem比对
bwa mem b73--ref b97_peaks.fa > b97.bwa.sam
#提取unmmaped的query seq
grep -v @ b97.bwa.sam | awk '$2==4 {print$1}'> b97.bwa.unmapped
#构造用来提取unmapped reads序列的bed文件
awk -F ":|-" '{print $1,$2,$3}' OFS="\t" b97.bwa.unmapped > b97.bwa.unmapped.bedinterm
sed 's/b97//' b97.bwa.unmapped.bedinterm > b97.bwa.unmapped.bed
#提取unmapped reads序列 给fa文件header加亲本信息
bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b97/Zm-B97-REFERENCE-NAM-1.0.fa -bed b97.bwa.unmapped.bed -fo b97.bwa.unmapped.woparent.fa
sed 's/>/&b97/' b97.bwa.unmapped.woparent.fa > b97.bwa.unmapped.fa
#加到ref的fa里面继续下一轮比对
cat b97.bwa.unmapped.fa ../fasta_peaks/b73_peaks.fa > b97--ref.fa

bwa index b97--ref.fa -p b97--ref
bwa mem b97--ref ../../fasta_peaks/cml52_peaks.fa > ../cml52.bwa.sam
grep -v @ cml52.bwa.sam | awk '$2==4 {print$1}' > cml52.bwa.unmapped
awk -F ":|-" '{print $1,$2,$3}' OFS="\t" cml52.bwa.unmapped > cml52.bwa.unmapped.bedinterm
sed 's/cml52//' cml52.bwa.unmapped.bedinterm > cml52.bwa.unmapped.bed
bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/cml52/Zm-CML52-REFERENCE-NAM-1.0.fa -bed cml52.bwa.unmapped.bed -fo ./fa_and_index/cml52.bwa.unmapped.woparent.fa
sed 's/>/&cml52/' ./fa_and_index/cml52.bwa.unmapped.woparent.fa > ./fa_and_index/cml52.bwa.unmapped.fa
cat ./fa_and_index/cml52.bwa.unmapped.fa ./fa_and_index/b97--ref.fa > ./fa_and_index/cml52--ref.fa


--
#windows下excel创造的config.txt每行末有\r，需要先去除
cat ../../panocr-iterate-config.txt |tr '\r' ' ' > ../../iterate-config-ref-query--.txt
rm ../../panocr-iterate-config.txt

###---------------------------循环 bwa_iterate.sh------------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 20
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/bwamem_iterate_26nam.log
source activate bwa
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/bwa_trial1/fa_and_index
cat ../../iterate-config-ref-query--.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    bwa index ${ref}--ref.fa -p ${ref}--ref
    bwa mem ${ref}--ref ../../fasta_peaks/${query}_peaks.fa > ../${query}.bwa.sam
    grep -v @ ../${query}.bwa.sam | awk '$2==4 {print$1}' > ../${query}.bwa.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../${query}.bwa.unmapped > ../${query}.bwa.unmapped.bedinterm
    bash ./sed_bedinterm2bed.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM-*.fa -bed ../${query}.bwa.unmapped.bed -fo ${query}.bwa.unmapped.woparent.fa
    bash ./sed_add_fa_parent.sh
    cat ${query}.bwa.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
###---------------------------循环 bwa_iterate.sh------------------------###


##-----------sed_bedinterm2bed.sh 目的：b97chr1 374056  374257  →  chr1 374056  374257--------###
#!/bin/bash
sed 's/b97//' ../b97.bwa.unmapped.bedinterm > ../b97.bwa.unmapped.bed
sed 's/cml52//' ../cml52.bwa.unmapped.bedinterm > ../cml52.bwa.unmapped.bed
...

###-----------sed_add_fa_parent.sh 目的：>chr1:374056-374257  →  >b97chr1:374056-374257-------###
#!/bin/bash
sed 's/>/&b97/' b97.bwa.unmapped.woparent.fa > b97.bwa.unmapped.fa
sed 's/>/&cml52/' cml52.bwa.unmapped.woparent.fa > cml52.bwa.unmapped.fa
...



###---------------------------循环 bwa_iterate2.sh------------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 20
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/bwamem_iterate_26nam_2.log
source activate bwa
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/bwa_trial2/fa_and_index2
cat ../../iterate-config-ref-query--2.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    bwa index ${ref}--ref.fa -p ${ref}--ref
    bwa mem ${ref}--ref ../../fasta_peaks/${query}_peaks.fa > ../${query}.bwa.sam
    grep -v @ ../${query}.bwa.sam | awk '$2==4 {print$1}' > ../${query}.bwa.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../${query}.bwa.unmapped > ../${query}.bwa.unmapped.bedinterm
    bash ./sed_bedinterm2bed2.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM-*.fa -bed ../${query}.bwa.unmapped.bed -fo ${query}.bwa.unmapped.woparent.fa
    bash ./sed_add_fa_parent2.sh
    cat ${query}.bwa.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
###---------------------------循环 bwa_iterate2.sh------------------------###


###---------------------------循环 bwa_iterate3.sh------------顺序是peak数目由大到小------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 20
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/bwamem_iterate_26nam_3.log
source activate bwa
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/bwa_trial3/fa_and_index3
cat ../../iterate-config-ref-query--3.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    bwa index ${ref}--ref.fa -p ${ref}--ref
    bwa mem ${ref}--ref ../../fasta_peaks/${query}_peaks.fa > ../${query}.bwa.sam
    grep -v @ ../${query}.bwa.sam | awk '$2==4 {print$1}' > ../${query}.bwa.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ../${query}.bwa.unmapped > ../${query}.bwa.unmapped.bedinterm
    bash ./sed_bedinterm2bed3.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM-*.fa -bed ../${query}.bwa.unmapped.bed -fo ${query}.bwa.unmapped.woparent.fa
    bash ./sed_add_fa_parent3.sh
    cat ${query}.bwa.unmapped.fa ${ref}--ref.fa > ${query}--ref.fa
done
###---------------------------循环 bwa_iterate3.sh------------顺序是peak数目由大到小-------------###



#### 比较总结三种比对方法

In [ ]:
#计算panocr长度
awk -F ":|-" '{print $3-$2+1}' panocr_blast_iterate1 > panocr_length
awk '{sum += $1};END {print sum}' panocr_length
#将三种比对方法iterate1 2 3构建的panocr分别paste在一起
paste panocr_blast_iterate1 panocr_bwa_iterate1 panocr_minim_iterate1 > blast_bwa_minim_panocr1
paste panocr_blast_iterate2 panocr_bwa_iterate2 panocr_minim_iterate2 > blast_bwa_minim_panocr2
paste panocr_blast_iterate3 panocr_bwa_iterate3 panocr_minim_iterate3 > blast_bwa_minim_panocr3
#给绘制韦恩图文件加列名
sed "1i blast\tbwa\tminimap2" blast_bwa_minim_panocr3 > blast_bwa_minim_panocr3_1
#再用R包venn.diagram绘制


###最终决定使用minimap2(用新不用旧），迭代3（peak数由大到小）

### minimap2迭代构建，不只考虑unmapped，还有coverage或identity小于90%的

#### coverage identity综合分析脚本

In [ ]:
perl ../../../get_the_detailed_mapping_information_from_bwa_bam_jymodified.pl ./bam/b97.minim.bam 1 > b97.bam.detail  ##质量值设置的是1
#发现flag=256或272的比对质量都是0

python ./script_calculate_cov_ide/calculate_bam_detail_coverage_identity.py


#########-----------循环minimap2-iterate3-not-only-unmapped.sh------------##########
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 50
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/not_only_unmapped_iterate3.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/not_only_unmapped
cat ../iterate-config-ref-query--3.txt | while read id
do
    arr=($id)
    ref=${arr[0]}
    query=${arr[1]}
    minimap2 -t 48 -a ./ref--fa/${ref}--ref.fa ../fasta_peaks/${query}_peaks.fa | samtools view -b -o ./bam/${query}.minim.bam
    samtools view ./bam/${query}.minim.bam | awk '$2==4 {print $1}' > ./unmapped/${query}.minim.unmapped
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ./unmapped/${query}.minim.unmapped > ./unmapped/${query}.minim.unmapped.bedinterm
    bash ./sed_bedinterm2bed3_1.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM*.fa -bed ./unmapped/${query}.minim.unmapped.bed -fo ./unmapped/${query}.minim.unmapped.woparent.fa
    bash ./sed_add_fa_parent3_1.sh
    perl ../../../get_the_detailed_mapping_information_from_minimap2_bam_jymodified.pl ./bam/${query}.minim.bam 10 > ./bam/${query}.q10.minim.bam.detail
    python ./calculate_bam_detail_coverage_and_identity.py ${query}
    grep True ./calculate_out/${query}.cov_and_ide.lessthan90.csv | awk -F ',' '{print $1}' > ./calculate_out/${query}.cov_and_ide.lessthan90.id
    awk -F ":|-" '{print $1,$2,$3}' OFS="\t" ./calculate_out/${query}.cov_and_ide.lessthan90.id > ./calculate_out/${query}.cov_and_ide.lessthan90.bedinterm
    bash ./sed_bedinterm2bed3_2.sh
    bedtools getfasta -fi /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${query}/*-REFERENCE-NAM*.fa -bed ./calculate_out/${query}.cov_and_ide.lessthan90.bed -fo ./calculate_out/${query}.cov_and_ide.lessthan90.woparent.fa
    bash ./sed_add_fa_parent3_2.sh
    cat ./unmapped/${query}.minim.unmapped.fa ./calculate_out/${query}.cov_and_ide.lessthan90.fa ./ref--fa/${ref}--ref.fa > ./ref--fa/${query}--ref.fa
done
#########-----------循环minimap2-iterate3-not-only-unmapped.sh------------##########

### CD-Hit-est聚类

In [ ]:
source activate jy_atac
conda install -c bioconda cd-hit -y
cat *_peaks.fa > allnam26peaks.fa


26个nam亲本所有peaks cat到一起做cluster

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 41
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/cd-hit-cluster.log
source activate jy_atac
cd-hit-est -i allnam26peaks.fa -o cluster_90 -c 0.9 -n 9 -M 0 -T 40 -d 0 -g 1

b73 ref去冗余

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 41
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/log/cd-hit-redundancy_elimi.log
source activate jy_atac
cd-hit-est -i b73_peaks.fa -o redundancy_elimi_b73 -c 0.9 -n 9 -M 0 -T 40 -d 0 -g 1


##路径  /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/fasta_peaks/cd-hit

### 测试部分亲本call出的peak数是否已经饱和

In [ ]:
#以cml103为例

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 20
#SBATCH -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/log/cml228_sample.log
source activate jy_atac
cd /public1/home/sc80041/yjiang22/nam/fastq/
ls ./cml228 | while read id
do
    for i in {1..9}
    do 
        seqtk sample -s100 ./cml228/${id} 0.$i > /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0/${id}
    done
done

###--------------------cml228 sample mapping QC merge peakcalling----------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 61
#SBATCH -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/log/cml228_sample_atacpipe.log
source activate jy_atac
cd /public1/home/sc80041/yjiang22/software/ATAC-pipe-master
for i in {1..9}
do
python atac-pipe_NAM_mt.py --MappingQC -t 60 -i /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0/ -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0 -r cml228

python atac-pipe_NAM_mt.py \
--Merge \
--bam /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0/Mapping \
-r cml228 \
-o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0 \
--group /public1/home/sc80041/yjiang22/nam/group/cml228_group

python atac-pipe_NAM_mt.py \
--PeakCalling \
--bed /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0/Mapping \
-r cml228 \
-o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/cml228/sample${i}0 \
--group /public1/home/sc80041/yjiang22/nam/group/cml228_group \
--project cml228

done


###################b73 sample########################

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/log/b73_sample80_mapping.log
source activate jy_atac
python atac-pipe_NAM_mt.py --MappingQC -t 95 -i /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_1/ -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_1 -r b73
python atac-pipe_NAM_mt.py --MappingQC -t 95 -i /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_2/ -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_2 -r b73
python atac-pipe_NAM_mt.py --MappingQC -t 95 -i /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_3/ -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_3 -r b73
python atac-pipe_NAM_mt.py --MappingQC -t 95 -i /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_4/ -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80_4 -r b73

###--------------------qukunlab_atac_pipe merge----------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/log/b73_sample80_merge_and_peakcalling.log
source activate jy_atac
python atac-pipe_NAM_mt.py \
--Merge \
--bam /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80/Mapping \
-r b73 \
-o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80 \
--group /public1/home/sc80041/yjiang22/nam/group/b73_group

###--------------------qukunlab_atac_pipe peak calling----------------------###

python atac-pipe_NAM_mt.py \
--PeakCalling \
--bed /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80/Mapping \
-r b73 \
-o /public1/home/sc80041/yjiang22/nam/peak_saturation_test/b73/sample80 \
--group /public1/home/sc80041/yjiang22/nam/group/b73_group \
--project b73



### 将其他亲本atac数据map到b73上，看mapping bias情况 (测试了oh43 cml228 mo18w tzi8)

In [ ]:
###-------------------------------以oh43为例--------------------------------###
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/out/mapping_bias_test/oh43_map_to_b73_bias.log
source activate jy_atac
python atac-pipe_NAM_mt.py \
--MappingQC \
-t 95 \
-i /public1/home/sc80041/yjiang22/nam/fastq/oh43/ \
-o /public1/home/sc80041/yjiang22/nam/out/mapping_bias_test/oh43 \
-r b73

python atac-pipe_NAM_mt.py \
--Merge \
--bam /public1/home/sc80041/yjiang22/nam/out/mapping_bias_test/oh43/Mapping \
-r b73 \
-o /public1/home/sc80041/yjiang22/nam/out/mapping_bias_test/oh43 \
--group /public1/home/sc80041/yjiang22/nam/group/oh43_group

python atac-pipe_NAM_mt.py \
--PeakCalling \
--bed /public1/home/sc80041/yjiang22/nam/out/mapping_bias_test/oh43/Mapping \
-r b73 \
-o /public1/home/sc80041/yjiang22/nam/out/mapping_bias_test/oh43 \
--group /public1/home/sc80041/yjiang22/nam/group/oh43_group \
--project oh43



### 用minimap2比对的方法，划分ocrl类型 （panoccr做query)

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 50
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/cluster_minimap2/log/b73_as_ref.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/cluster_minimap2
minimap2 -t 48 -a ../fasta_peaks/b73_peaks.fa ./panocr_withid.fa | samtools view -b -o ./bam/b73_as_ref.bam


perl ../../../get_the_detailed_mapping_information_from_minimap2_bam_jymodified.pl ./bam/b73_as_ref.bam 10 > ./bam/b73_as_ref.bam.q10.detail
python calculate_bam_detail_coverage_and_identity.py b73
grep True calculate_out/b73.cov_and_ide.morethan90.csv | awk -F ',' '{print $1}' > ./calculate_out/b73_as_ref.cov_and_ide.morethan90.id


#出现了270条，同一条序列互相比对  全match但是被质量值q10过滤掉的情况，或unmapped的情况，
#导致一些panocr在所有亲本中都找不到
#解决方案：perl脚本质量值阈值设置为0，对于unmapped根据panocr id手动修改存在于哪个亲本

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 50
#SBATCH -o /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/cluster_minimap2/log/each_parent_peaks_fa_as_ref.log
source activate /public1/home/sc80041/miniconda3/envs/minimap2
cd /public1/home/sc80041/yjiang22/nam/pan_ocr_construction/ocr_iterate/cluster_minimap2
cat ../../../nam26_lowercase | while read id
do
    minimap2 -t 48 -a ../fasta_peaks/${id}_peaks.fa ./panocr_withid.fa | samtools view -b -o ./bam/${id}_as_ref.bam
    perl ../../../get_the_detailed_mapping_information_from_minimap2_bam_jymodified.pl ./bam/${id}_as_ref.bam 10 > ./bam/${id}_as_ref.bam.q10.detail
    python calculate_bam_detail_coverage_and_identity.py ${id}
    grep True calculate_out/${id}.cov_and_ide.morethan90.csv | awk -F ',' '{print $1}' > ./calculate_out/${id}_as_ref.cov_and_ide.morethan90.id
done




### lyt b73 seedling frozen atac数据比对到b73 v5基因组，看与我b73 v5 call出的10万多个peak overlap情况如何

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -n 96
#SBATCH -o /public1/home/sc80041/yjiang22/lyt/b73seedlingfrozen_ATAC-pipe.log
source activate jy_atac
cd /public1/home/sc80041/yjiang22/software/ATAC-pipe-master

python atac-pipe_NAM_mt.py --MappingQC -t 95 -i /public1/home/sc80041/yjiang22/lyt/fastq/fastq1/ -o /public1/home/sc80041/yjiang22/lyt/ -r b73
python atac-pipe_NAM_mt.py --MappingQC -t 95 -i /public1/home/sc80041/yjiang22/lyt/fastq/fastq2/ -o /public1/home/sc80041/yjiang22/lyt/ -r b73

python atac-pipe_NAM_mt.py \
--Merge \
--bam /public1/home/sc80041/yjiang22/lyt/Mapping \
-r b73 \
-o /public1/home/sc80041/yjiang22/lyt/ \
--group /public1/home/sc80041/yjiang22/lyt/b73_group

python atac-pipe_NAM_mt.py \
--PeakCalling \
--bed /public1/home/sc80041/yjiang22/lyt/Mapping \
-r b73 \
-o /public1/home/sc80041/yjiang22/lyt/ \
--group /public1/home/sc80041/yjiang22/lyt/b73_group \
--project b73



bedtools intersect -a jy_b73v5_narrowpeak -b lyt_b73v5_narrowpeak -wao > intersect_wao


### 鉴定基因组之间的变异（snp, indel, sv)
流程采用 nucmer + assemblytics,没用maxmatch参数，用的默认参数
sv: nucmer + assemblytics + survivor  ;
snp: nucmer + dnadiff

### sv

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference/log/b73-cml103-nucmer.log
source activate mummer4
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference
nucmer -l 100 -c 500 -t 30 /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b73/Zm-B73-REFERENCE-NAM-5.0.fa /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/cml103/Zm-CML103-REFERENCE-NAM-1.0.fa --prefix b73-cml103

#弃用网页端的assemblytics，安装命令行的assemblytics,再安装依赖的r包和python包
#单个为例 assemblytics
nohup /public1/home/sc80041/yjiang22/software/Assemblytics-master/scripts/Assemblytics ./p39_nucmer/b73-p39.delta b73-p39 10000 1 10000 &
#批量asssemblytics
ls | cut -d '.' -f 1 | uniq | while read id;do /public1/home/sc80041/yjiang22/software/Assemblytics-master/scripts/Assemblytics ./${id}_nucmer/${id}.delta ${id} 10000 1 10000;done

#处理assemblytics结果文件
#SURVIVOR convertAssemblytics（将assemblytics的结果bed文件转为vcf文件，并分成insertion和deletion两个文件，再将25*2个vcf文件merge）

#!/bin/bash
source activate gan-survivor
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference/assemblytics
ls | while read id
do
    SURVIVOR convertAssemblytics ./${id}/b73-${id}.Assemblytics_structural_variants.bed 50 ./${id}/b73-${id}.Assemblytics.SVs.vcf
    grep '<DEL>' ./${id}/b73-${id}.Assemblytics.SVs.vcf > ./${id}/b73-${id}.Assemblytics.del.vcf
    grep '<INS>' ./${id}/b73-${id}.Assemblytics.SVs.vcf > ./${id}/b73-${id}.Assemblytics.ins.vcf
    sort -k1,1 -k2,2n ./${id}/b73-${id}.Assemblytics.ins.vcf > ./${id}/b73-${id}.Assemblytics.ins.sorted.vcf
    sort -k1,1 -k2,2n ./${id}/b73-${id}.Assemblytics.del.vcf > ./${id}/b73-${id}.Assemblytics.del.sorted.vcf
done

#!/bin/bash
source activate gan-survivor
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref/assemblytics
SURVIVOR merge b73-as-ref.ins.vcf.namelist 1000 1 1 1 1 50 b73-as-ref.merged.ins.vcf ##将上面*ins.sorted.vcf文件路径存入到b73-as-ref.ins.vcf.namelist
SURVIVOR merge b73-as-ref.del.vcf.namelist 1000 1 1 1 1 50 b73-as-ref.merged.del.vcf ##将上面*del.sorted.vcf文件路径存入到b73-as-ref.del.vcf.namelist

grep -v '#' b73-as-ref.merged.ins.vcf | awk -F ';| ' '{print $1,$2,$7}' | awk -F ' ' '{print $1,$2,$10}' | sed 's/END=//g' | tr ' ' '\t' > b73-as-ref.merged.ins.vcf.bed
grep -v '#' b73-as-ref.merged.del.vcf | awk -F ';| ' '{print $1,$2,$7}' | awk -F ' ' '{print $1,$2,$10}' | sed 's/END=//g' | tr ' ' '\t' > b73-as-ref.merged.del.vcf.bed
#再去除上面bed文件里面scaf
sed -i '/scaf/d' b73-as-ref.merged.del.vcf.bed
sed -i '/scaf/d' b73-as-ref.merged.ins.vcf.bed

#### panocr b73 subset里各类ocr的坐标

In [ ]:

grep b73 cluster_dataframe.csv | grep -w '\<core\>' > b73_core.csv
grep b73 cluster_dataframe.csv | grep private > b73_private.csv
grep b73 cluster_dataframe.csv | grep softcore > b73_softcore.csv
grep b73 cluster_dataframe.csv | grep dispensable > b73_dispensable.csv

awk -F ':|-|,|_' '{print $2,$3,$4}' b73_dispensable.csv | sed 's/b73//g' | tr ' ' '\t' > b73_dispensable.bed
awk -F ':|-|,|_' '{print $2,$3,$4}' b73_core.csv | sed 's/b73//g' | tr ' ' '\t' > b73_core.bed
awk -F ':|-|,|_' '{print $2,$3,$4}' b73_softcore.csv | sed 's/b73//g' | tr ' ' '\t' > b73_softcore.bed
awk -F ':|-|,|_' '{print $2,$3,$4}' b73_private.csv | sed 's/b73//g' | tr ' ' '\t' > b73_private.bed



#### sv坐标与core dispensable ...ocr取交集，对于insertion只要有1bp重叠则count，对于deletion两者有50% overlap则count

In [ ]:
bedtools intersect -a b73_core.bed -b assemblytics/b73-as-ref.merged.del.vcf.bed -wao > b73_core.sv_del.wao
bedtools intersect -a b73_core.bed -b assemblytics/b73-as-ref.merged.ins.vcf.bed -wao > b73_core.sv_ins.wao

### snp

In [ ]:
##----dnadiff找变异

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 10
#SBATCH -o /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference/log/b73-as-ref-dnadiff.log
source activate mummer4
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference
cut -f 2 iterate-config-ref-query--3.txt | while read id
do
    dnadiff -d b73-${id}.delta -p b73-${id} && echo "---b73-${id} dnadiff done---"
done

##----转换dnadiff结果的.snp文件
/public1/home/sc80041/yjiang22/software/all2vcf-master/all2vcf mummer --snps ./oh43_nucmer/b73-oh43.snps --reference /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b73/Zm-B73-REFERENCE-NAM-5.0.fa --type ALL > ./oh43_nucmer/b73-oh43.snps.all2vcf.vcf

#!/bin/bash
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference/
cut -f 2 iterate-config-ref-query--3.txt | while read id
do 
    /public1/home/sc80041/yjiang22/software/all2vcf-master/all2vcf mummer \
    --snps ./${id}/b73-${id}.snps \
    --reference /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b73/Zm-B73-REFERENCE-NAM-5.0.fa \
    --type  ALL > ./${id}/b73-${id}.snps.all2vcf.vcf
done

##----提取all2vcf.vcf里面的snp
cut -f 2 iterate-config-ref-query--3.txt | while read id
do 
    awk '$8=="." {print $0}' ./${id}_nucmer/b73-${id}.snps.all2vcf.vcf > ./${id}_nucmer/b73-${id}.snps.all2vcf.vcf.snp
done
cut -f 1,2 */*all2vcf.vcf.snp >> b73-as-ref.snp
grep -v 'scaf' b73-as-ref.snp > b73-as-ref.chr.snp
cat b73-as-ref.chr.snp | sort | uniq > b73-as-ref.chr.snp.uniq
sort -k1,1 -k2,2n b73-as-ref.chr.snp.uniq > b73-as-ref.chr.snp.uniq.sort
cut -f 2 b73-as-ref.chr.snp.uniq.sort > tmp
paste b73-as-ref.chr.snp.uniq.sort tmp > b73-as-ref.chr.snp.uniq.sort.bed
bedtools intersect -a b73_core.bed -b b73-as-ref.chr.snp.uniq.sort.bed -wao > b73_core.snp.wao
bedtools intersect -a b73_softcore.bed -b b73-as-ref.chr.snp.uniq.sort.bed -wao > b73_softcore.snp.wao
bedtools intersect -a b73_dispensable.bed -b b73-as-ref.chr.snp.uniq.sort.bed -wao > b73_dispensable.snp.wao
bedtools intersect -a b73_private.bed -b b73-as-ref.chr.snp.uniq.sort.bed -wao > b73_private.snp.wao

grep -v '\.' b73_core.snp.wao | wc -l



#### indel(1-50bp)

In [ ]:
cut -f 2 iterate-config-ref-query--3.txt | while read id;do grep 'INDEL' ./${id}_nucmer/*.snps.all2vcf.vcf > ${id}_nucmer/b73-${id}.snps.all2vcf.vcf.indel;done
cut -f 1,2 */*snps.all2vcf.vcf.indel >> b73-as-ref.indel
grep -v 'scaf' b73-as-ref.indel > b73-as-ref.chr.indel
cat b73-as-ref.chr.indel | sort | uniq > b73-as-ref.chr.indel.uniq
sort -k1,1 -k2,2n b73-as-ref.chr.indel.uniq > b73-as-ref.chr.indel.uniq.sort
cut -f 2 b73-as-ref.chr.indel.uniq.sort > tmp
paste b73-as-ref.chr.indel.uniq.sort tmp > b73-as-ref.chr.indel.uniq.sort.bed
rm tmp
bedtools intersect -a b73_core.bed -b b73-as-ref.chr.indel.uniq.sort.bed -wao > b73_core.indel.wao
grep -v '\.' b73_core.indel.wao | wc -l





### oh43划分四类ocr，依据先前panocr做query，各亲本（含oh43）peaks.fa做ref，minimap2比对的结果，保留one-to-one，cov_and_ide>=90%，建立已划分好label的panocr与oh43 ocr之间一对一的关系，以划分oh43 ocr的类别

#### panocr_oh43_subset 遗传变异特征（与b73 subset类似）

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/variation_detection/oh43_as_ref_mumreference/log/oh43-as-ref-nucmer-dnadiff.log
source activate mummer4
cd /public1/home/sc80041/yjiang22/nam/variation_detection/oh43_as_ref_mumreference
cut -f 2 query_list | while read id
do
    nucmer -l 100 -c 500 -t 30 /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/oh43/Zm-Oh43-REFERENCE-NAM-1.0.fa /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${id}/*NAM*.fa --prefix oh43-${id} && echo "---oh43-${id} nucmer done---"
    dnadiff -d oh43-${id}.delta -p oh43-${id} && echo "---oh43-${id} dnadiff done---"
done

##--------------- sv ------------------##
ls | cut -d '.' -f 1 | uniq | while read id
do 
    /public1/home/sc80041/yjiang22/software/Assemblytics-master/scripts/Assemblytics ./${id}_nucmer/${id}.delta ${id} 10000 1 10000
done

#!/bin/bash
source activate gan-survivor
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference/assemblytics
ls | while read id
do
    SURVIVOR convertAssemblytics ./${id}/oh43-${id}.Assemblytics_structural_variants.bed 50 ./${id}/oh43-${id}.Assemblytics.SVs.vcf
    grep '<DEL>' ./${id}/oh43-${id}.Assemblytics.SVs.vcf > ./${id}/oh43-${id}.Assemblytics.del.vcf
    grep '<INS>' ./${id}/oh43-${id}.Assemblytics.SVs.vcf > ./${id}/oh43-${id}.Assemblytics.ins.vcf
    sort -k1,1 -k2,2n ./${id}/oh43-${id}.Assemblytics.ins.vcf > ./${id}/oh43-${id}.Assemblytics.ins.sorted.vcf
    sort -k1,1 -k2,2n ./${id}/oh43-${id}.Assemblytics.del.vcf > ./${id}/oh43-${id}.Assemblytics.del.sorted.vcf
done

#!/bin/bash
source activate gan-survivor
cd /public1/home/sc80041/yjiang22/nam/variation_detection/oh43_as_ref_mumreference/assemblytics
SURVIVOR merge oh43-as-ref.ins.vcf.namelist 1000 1 1 1 1 50 oh43-as-ref.merged.ins.vcf ##将上面*ins.sorted.vcf文件路径存入到b73-as-ref.ins.vcf.namelist
SURVIVOR merge oh43-as-ref.del.vcf.namelist 1000 1 1 1 1 50 oh43-as-ref.merged.del.vcf ##将上面*del.sorted.vcf文件路径存入到b73-as-ref.del.vcf.namelist

grep -v '#' oh43-as-ref.merged.ins.vcf | awk -F ';| ' '{print $1,$2,$7}' | awk -F ' ' '{print $1,$2,$10}' | sed 's/END=//g' | tr ' ' '\t' > oh43-as-ref.merged.ins.vcf.bed
grep -v '#' oh43-as-ref.merged.del.vcf | awk -F ';| ' '{print $1,$2,$7}' | awk -F ' ' '{print $1,$2,$10}' | sed 's/END=//g' | tr ' ' '\t' > oh43-as-ref.merged.del.vcf.bed
#再去除上面bed文件里面scaf
sed -i '/scaf/d' oh43-as-ref.merged.del.vcf.bed
sed -i '/scaf/d' oh43-as-ref.merged.ins.vcf.bed

##--------------- snp ------------------##

##----转换dnadiff结果的.snp文件
/public1/home/sc80041/yjiang22/software/all2vcf-master/all2vcf mummer --snps ./oh43_nucmer/b73-oh43.snps --reference /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b73/Zm-B73-REFERENCE-NAM-5.0.fa --type ALL > ./oh43_nucmer/b73-oh43.snps.all2vcf.vcf

#!/bin/bash
cd /public1/home/sc80041/yjiang22/nam/variation_detection/b73_as_ref_mumreference/
cat query_list | while read id
do 
    /public1/home/sc80041/yjiang22/software/all2vcf-master/all2vcf mummer \
    --snps ./${id}_nucmer/oh43-${id}.snps \
    --reference /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b73/Zm-Oh43-REFERENCE-NAM-1.0.fa \
    --type  ALL > ./${id}/oh43-${id}.snps.all2vcf.vcf
done

##----提取all2vcf.vcf里面的snp
cat query_list | while read id;do awk '$8=="." {print $0}' ./${id}_nucmer/oh43-${id}.snps.all2vcf.vcf > ./${id}_nucmer/oh43-${id}.snps.all2vcf.vcf.snp;done
cut -f 1,2 */*all2vcf.vcf.snp >> oh43-as-ref.snp
grep -v 'scaf' oh43-as-ref.snp > oh43-as-ref.chr.snp
cat oh43-as-ref.chr.snp | sort | uniq > oh43-as-ref.chr.snp.uniq
sort -k1,1 -k2,2n oh43-as-ref.chr.snp.uniq > oh43-as-ref.chr.snp.uniq.sort
cut -f 2 oh43-as-ref.chr.snp.uniq.sort > tmp
paste oh43-as-ref.chr.snp.uniq.sort tmp > oh43-as-ref.chr.snp.uniq.sort.bed

bedtools intersect -a oh43_core.bed -b oh43-as-ref.chr.snp.uniq.sort.bed -wao > oh43_core.snp.wao
bedtools intersect -a oh43_dispensable.bed -b oh43-as-ref.chr.snp.uniq.sort.bed -wao > oh43_dispensable.snp.wao
bedtools intersect -a oh43_softcore.bed -b oh43-as-ref.chr.snp.uniq.sort.bed -wao > oh43_softcore.snp.wao
bedtools intersect -a oh43_private.bed -b oh43-as-ref.chr.snp.uniq.sort.bed -wao > oh43_private.snp.wao

grep -v '\.' oh43_core.snp.wao | wc -l


##------------- indel(1-50bp) ----------##
cat query_list | while read id;do grep 'INDEL' ./${id}_nucmer/*.snps.all2vcf.vcf > ${id}_nucmer/oh43-${id}.snps.all2vcf.vcf.indel;done
cut -f 1,2 *_nucmer/*snps.all2vcf.vcf.indel >> oh43-as-ref.indel
grep -v 'scaf' oh43-as-ref.indel > oh43-as-ref.chr.indel
cat oh43-as-ref.chr.indel | sort | uniq > oh43-as-ref.chr.indel.uniq
sort -k1,1 -k2,2n oh43-as-ref.chr.indel.uniq > oh43-as-ref.chr.indel.uniq.sort
cut -f 2 oh43-as-ref.chr.indel.uniq.sort > tmp
paste oh43-as-ref.chr.indel.uniq.sort tmp > oh43-as-ref.chr.indel.uniq.sort.bed
bedtools intersect -a oh43_core.bed -b oh43-as-ref.chr.indel.uniq.sort.bed -wao > oh43_core.indel.wao
bedtools intersect -a oh43_private.bed -b oh43-as-ref.chr.indel.uniq.sort.bed -wao > oh43_private.indel.wao
bedtools intersect -a oh43_softcore.bed -b oh43-as-ref.chr.indel.uniq.sort.bed -wao > oh43_softcore.indel.wao
bedtools intersect -a oh43_dispensable.bed -b oh43-as-ref.chr.indel.uniq.sort.bed -wao > oh43_dispensable.indel.wao

grep -v '\.' oh43_core.indel.wao | wc -l
grep -v '\.' oh43_softcore.indel.wao | wc -l
grep -v '\.' oh43_dispensable.indel.wao | wc -l
grep -v '\.' oh43_private.indel.wao | wc -l



#### panocr_tzi8_subset 遗传变异特征（与b73 subset类似）

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 60
#SBATCH -o /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference/log/tzi8-as-ref-nucmer.log
source activate mummer4
cd /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference
cat query_list | while read id
do
    nucmer -l 100 -c 500 -t 60 /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/tzi8/Zm-Tzi8-REFERENCE-NAM-1.0.fa /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/${id}/*NAM*.fa --prefix tzi8-${id} && echo "---tzi8-${id} nucmer done---"
done

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 5
#SBATCH -o /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference/log/tzi8-as-ref-dnadiff.log
source activate mummer4
cd /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference
cat query_list | while read id
do
    dnadiff -d tzi8-${id}.delta -p tzi8-${id} && echo "---tzi8-${id} dnadiff done---"
done

##--------------- sv ------------------##
#!/bin/bash
cd /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference/assemblytics
cat ../query_list | while read id
do 
    /public1/home/sc80041/yjiang22/software/Assemblytics-master/scripts/Assemblytics ./${id}/tzi8-${id}.delta ${id} 10000 1 10000
done

#!/bin/bash
source activate gan-survivor
cd /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference/assemblytics
cat ../query_list | while read id
do
    SURVIVOR convertAssemblytics ./${id}/${id}.Assemblytics_structural_variants.bed 50 ./${id}/${id}.Assemblytics.SVs.vcf
    grep '<DEL>' ./${id}/${id}.Assemblytics.SVs.vcf > ./${id}/${id}.Assemblytics.del.vcf
    grep '<INS>' ./${id}/${id}.Assemblytics.SVs.vcf > ./${id}/${id}.Assemblytics.ins.vcf
    sort -k1,1 -k2,2n ./${id}/${id}.Assemblytics.ins.vcf > ./${id}/${id}.Assemblytics.ins.sorted.vcf
    sort -k1,1 -k2,2n ./${id}/${id}.Assemblytics.del.vcf > ./${id}/${id}.Assemblytics.del.sorted.vcf
done

#!/bin/bash
source activate gan-survivor
cd /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference/assemblytics
SURVIVOR merge tzi8-as-ref.ins.vcf.namelist 1000 1 1 1 1 50 tzi8-as-ref.merged.ins.vcf ##将上面*ins.sorted.vcf文件路径存入到b73-as-ref.ins.vcf.namelist
SURVIVOR merge tzi8-as-ref.del.vcf.namelist 1000 1 1 1 1 50 tzi8-as-ref.merged.del.vcf ##将上面*del.sorted.vcf文件路径存入到b73-as-ref.del.vcf.namelist

grep -v '#' tzi8-as-ref.merged.ins.vcf | awk -F ';| ' '{print $1,$2,$7}' | awk -F ' ' '{print $1,$2,$10}' | sed 's/END=//g' | tr ' ' '\t' > tzi8-as-ref.merged.ins.vcf.bed
grep -v '#' tzi8-as-ref.merged.del.vcf | awk -F ';| ' '{print $1,$2,$7}' | awk -F ' ' '{print $1,$2,$10}' | sed 's/END=//g' | tr ' ' '\t' > tzi8-as-ref.merged.del.vcf.bed
#再去除上面bed文件里面scaf
sed -i '/scaf/d' tzi8-as-ref.merged.del.vcf.bed
sed -i '/scaf/d' tzi8-as-ref.merged.ins.vcf.bed

##--------------- snp ------------------##

##----转换dnadiff结果的.snp文件

#!/bin/bash
cd /public1/home/sc80041/yjiang22/nam/variation_detection/tzi8_as_ref_mumreference/
cat query_list | while read id
do 
    /public1/home/sc80041/yjiang22/software/all2vcf-master/all2vcf mummer \
    --snps ./${id}_nucmer/tzi8-${id}.snps \
    --reference /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/tzi8/Zm-Tzi8-REFERENCE-NAM-1.0.fa \
    --type  ALL > ./${id}_nucmer/tzi8-${id}.snps.all2vcf.vcf && echo "---${id} all2vcf done----"
done

##----提取all2vcf.vcf里面的snp
cat query_list | while read id;do awk '$8=="." {print $0}' ./${id}_nucmer/tzi8-${id}.snps.all2vcf.vcf > ./${id}_nucmer/tzi8-${id}.snps.all2vcf.vcf.snp;done
cut -f 1,2 *nucmer/*all2vcf.vcf.snp >> tzi8-as-ref.snp
grep -v 'scaf' tzi8-as-ref.snp > tzi8-as-ref.chr.snp
cat tzi8-as-ref.chr.snp | sort | uniq > tzi8-as-ref.chr.snp.uniq
sort -k1,1 -k2,2n tzi8-as-ref.chr.snp.uniq > tzi8-as-ref.chr.snp.uniq.sort
cut -f 2 tzi8-as-ref.chr.snp.uniq.sort > tmp
paste tzi8-as-ref.chr.snp.uniq.sort tmp > tzi8-as-ref.chr.snp.uniq.sort.bed
rm tmp
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_core.bed -b tzi8-as-ref.chr.snp.uniq.sort.bed -wao > tzi8_core.snp.wao
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_dispensable.bed -b tzi8-as-ref.chr.snp.uniq.sort.bed -wao > tzi8_dispensable.snp.wao
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_softcore.bed -b tzi8-as-ref.chr.snp.uniq.sort.bed -wao > tzi8_softcore.snp.wao
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_private.bed -b tzi8-as-ref.chr.snp.uniq.sort.bed -wao > tzi8_private.snp.wao
#计数
grep -v '\.' oh43_core.snp.wao | wc -l


##------------- indel(1-50bp) ----------##
cat query_list | while read id;do grep 'INDEL' ./${id}_nucmer/*.snps.all2vcf.vcf > ${id}_nucmer/tzi8-${id}.snps.all2vcf.vcf.indel;done
cut -f 1,2 *_nucmer/*snps.all2vcf.vcf.indel >> tzi8-as-ref.indel
grep -v 'scaf' tzi8-as-ref.indel > tzi8-as-ref.chr.indel
cat tzi8-as-ref.chr.indel | sort | uniq > tzi8-as-ref.chr.indel.uniq
sort -k1,1 -k2,2n tzi8-as-ref.chr.indel.uniq > tzi8-as-ref.chr.indel.uniq.sort
cut -f 2 tzi8-as-ref.chr.indel.uniq.sort > tmp
paste tzi8-as-ref.chr.indel.uniq.sort tmp > tzi8-as-ref.chr.indel.uniq.sort.bed
rm tmp
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_core.bed -b tzi8-as-ref.chr.indel.uniq.sort.bed -wao > tzi8_core.indel.wao
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_private.bed -b tzi8-as-ref.chr.indel.uniq.sort.bed -wao > tzi8_private.indel.wao
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_softcore.bed -b tzi8-as-ref.chr.indel.uniq.sort.bed -wao > tzi8_softcore.indel.wao
bedtools intersect -a ./tzi8_label_process/tzi8_panocr_dispensable.bed -b tzi8-as-ref.chr.indel.uniq.sort.bed -wao > tzi8_dispensable.indel.wao

grep -v '\.' tzi8_core.indel.wao | wc -l
grep -v '\.' tzi8_softcore.indel.wao | wc -l
grep -v '\.' tzi8_dispensable.indel.wao | wc -l
grep -v '\.' tzi8_private.indel.wao | wc -l





### 绘制ocr上snp变异趋势

In [ ]:
zcat all.chr.raw.vcf.gz | awk -F "\t" -v OFS="\t" '{print $1,$2,$3}' > B73.snp
grep -v "#" B73.snp |awk -F "\t" -v OFS="\t" '{print "chr"$1,$2,$2,$3}' >B73.snp.bed
bedToBam -i B73.snp.bed -g B73V4.genome >B73.snp.bam
samtools index B73.snp.bam
bamCoverage -o B73.snp.bw -b B73.snp.bam  -p 20 --effectiveGenomeSize 2100000000 --binSize 10
computeMatrix reference-point --referencePoint center -p 30 -b 2000 -a 2000 -R ear.bed leaf.bed tassel.bed seedling.bed common.bed -S B73.snp.bw  --skipZeros -o tissue_matrix_summit_2kb.gz
plotProfile -m tissue_matrix_summit_2kb.gz -out tissue_merged_matrix_summit_2kb.svg --perGroup #每个sample一个图
plotProfile -m tissue_matrix_summit_2kb.gz -out tissue_merged_matrix_summit_2kb.svg #所有的sample merged到一张


bedToBam -i b73.plot.snp.bed -g b73.genome > b73.snp.bam  ##b73.plot.snp.bed要求四列，chr start end id; b73.genome格式为chr \t length
samtools index b73.snp.bam
bamCoverage -o b73.snp.bw -b b73.snp.bam  -p 10 --effectiveGenomeSize 2100000000 --binSize 10
computeMatrix reference-point --referencePoint center -p 10 -b 2000 -a 2000 -R ../b73_private.bed ../b73_dispensable.bed ../b73_softcore.bed ../b73_core.bed -S b73.snp.bw  --skipZeros -o type4_matrix_summit_2kb.gz
plotProfile -m type4_matrix_summit_2kb.gz -out type4_matrix_summit_2kb.gz_per.svg --perGroup #每个sample一个图
plotProfile -m type4_matrix_summit_2kb.gz -out type4_matrix_summit_2kb.gz_merge.svg #所有的sample merged到一张



### EDTA注释转座子

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 30
#SBATCH -o /public1/home/sc80041/yjiang22/nam/TE/log/b73.edta.log
source activate EDTA
cd /public1/home/sc80041/yjiang22/nam/TE
perl /public1/home/sc80041/yjiang22/software/EDTA-master/EDTA.pl \
    -genome ./B73/Zm-B73-REFERENCE-NAM-5.0.fa \
    -step all -species Maize --curatedlib maizeTE02052020 \
    --cds ./B73/Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.cds.fa \
    --anno 1 \
    --evaluate 0\
     -t 30 \
    && echo "--b73 edta done--"



### 更新call peak流程

In [ ]:
#各亲本重复间_1.fq合并，_2.fq合并
cat b73rep*_1.fq >> b73_1.fq
cat b73rep*_2.fq >> b73_2.fq

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/fastq/b73/process.log
cd /public1/home/sc80041/yjiang22/nam/fastq/b73
source activate jy_atac
fastp --thread 80 -i b73_1.fq.gz -I b73_2.fq.gz -o b73_1.fastp.fq.gz -O b73_2.fastp.fq.gz --detect_adapter_for_pe --correction --length_required 35 && echo '--fastp done--'
bowtie2 -p 94 --local --very-sensitive-local --seed 1 -q --no-mixed --no-discordant --maxins 1000 \
    -x /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/b73/b73 \
    -1 b73_1.fastp.fq.gz -2 b73_2.fastp.fq.gz > b73.sam && echo '--bowtie2 done--'

samtools view -@ 80 -b -h -S b73.sam > b73.bam
samtools sort b73.bam -@ 80 -O bam -o b73.sort.bam
sambamba markdup --remove-duplicates b73.sort.bam b73.sort.rmdup.bam && echo '--rmdup done--' ##去除pcr重复
samtools view -@ 80 -q 30 -O bam -o b73.sort.rmdup.q30.bam b73.sort.rmdup.bam  ##mapping quality按30过滤
samtools index -@ 80 b73.sort.rmdup.q30.bam
alignmentSieve --numberOfProcessors 40 --ATACshift -b b73.sort.rmdup.q30.bam -o b73.sort.rmdup.q30.shift.bam && echo '--shift done--' ##移码
bedtools bamtobed -i b73.sort.rmdup.q30.shift.bam > b73.sort.rmdup.q30.shift.bed && echo '--bam2bed done--'
samtools sort b73.sort.rmdup.q30.shift.bam -@ 80 -O bam -o b73.sort.rmdup.q30.shift.sort.bam
samtools index -@ 80 b73.sort.rmdup.q30.shift.sort.bam
bamCoverage -o b73.sort.rmdup.q30.shift.sort.bw -b b73.sort.rmdup.q30.shift.sort.bam -p 40 --normalizeUsing RPGC --effectiveGenomeSize 2100000000 --binSize 10
macs2 callpeak --format BED --gsize 2.1e+9 --keep-dup all --qvalue 0.05 --extsize 200 --shift -100 --nomodel -t b73.sort.rmdup.q30.shift.bed -n b73 --outdir ./ && echo '--call peak done--'


macs2 callpeak --format BED --gsize 1.8e+9 --keep-dup all --qvalue 0.005 --extsize 200 --shift -100 --nomodel -t b73.sort.rmdup.q30.shift.bed -n b73 --outdir ../ 

In [ ]:
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 10
#SBATCH -o /public1/home/sc80041/yjiang22/nam/fastq/oh43/fastp.log
cd /public1/home/sc80041/yjiang22/nam/fastq/oh43
source activate jy_atac
fastp -i oh43_1.fq.gz -I oh43_2.fq.gz -o oh43_1.fastp.fq.gz -O oh43_2.fastp.fq.gz --detect_adapter_for_pe --correction --length_required 35

#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/fastq/oh43/bowtie2.log
cd /public1/home/sc80041/yjiang22/nam/fastq/oh43
source activate jy_atac
bowtie2 -p 94 --local --very-sensitive-local --seed 1 -q --no-mixed --no-discordant --maxins 1000 \
    -x /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/oh43/oh43 \
    -1 oh43_1.fastp.fq.gz -2 oh43_2.fastp.fq.gz > oh43.sam

samtools view -@ 8 -b -h -S oh43.sam > oh43.bam
samtools sort oh43.bam -@ 8 -O bam -o oh43.sort.bam
sambamba markdup --remove-duplicates oh43.sort.bam oh43.sort.rmdup.bam ##去除pcr重复
samtools view -@ 8 -q 30 -O bam -o oh43.sort.rmdup.q30.bam oh43.sort.rmdup.bam  ##mapping quality按30过滤
samtools index -@ 8 oh43.sort.rmdup.q30.bam
alignmentSieve --numberOfProcessors 8 --ATACshift -b oh43.sort.rmdup.q30.bam -o oh43.sort.rmdup.q30.shift.bam ##移码
bedtools bamtobed -i oh43.sort.rmdup.q30.shift.bam > oh43.sort.rmdup.q30.shift.bed
samtools sort oh43.sort.rmdup.q30.shift.bam -@ 8 -O bam -o oh43.sort.rmdup.q30.shift.sort.bam
samtools index -@ 8 oh43.sort.rmdup.q30.shift.sort.bam
bamCoverage -o oh43.sort.rmdup.q30.shift.sort.bw -b oh43.sort.rmdup.q30.shift.sort.bam -p 8 --normalizeUsing RPGC --effectiveGenomeSize 2100000000 --binSize 10
macs2 callpeak --format BED --gsize 2.1e+9 --keep-dup all --qvalue 0.05 --extsize 200 --shift -100 --nomodel -t oh43.sort.rmdup.q30.shift.bed -n oh43 --outdir ./



In [ ]:
#########-----------------fastp.sh-------#############
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 10
#SBATCH -o /public1/home/sc80041/yjiang22/nam/new_pipe_to_rm_FP/fastp.log
source activate jy_atac
sample_name=$1
cd /public1/home/sc80041/yjiang22/nam/new_pipe_to_rm_FP/$1
fastp -i $1_1.fq.gz -I $1_2.fq.gz -o $1_1.fastp.fq.gz -O $1_2.fastp.fq.gz --detect_adapter_for_pe --correction --length_required 35 && echo '--$1 fastp done--'
##uasge example sbatch ./fastp.sh tzi8


#########-----------------peakcalling.sh-------#############
#!/bin/bash
#SBATCH -p v6_384
#SBATCH -N 1
#SBATCH -c 96
#SBATCH -o /public1/home/sc80041/yjiang22/nam/new_pipe_to_rm_FP/peakcalling.log
sample_name=$1
source activate jy_atac
cd /public1/home/sc80041/yjiang22/nam/new_pipe_to_rm_FP/$1
bowtie2 -p 95 --local --very-sensitive-local --seed 1 -q --no-mixed --no-discordant --maxins 1000 \
    -x /public1/home/sc80041/yjiang22/software/ATAC-pipe-master/Data/Ref/$1/$1 \
    -1 $1_1.fastp.fq.gz -2 $1_2.fastp.fq.gz > $1.sam

samtools view -@ 80 -b -h -S $1.sam > $1.bam
rm $1.sam
samtools sort -@ 80 $1.bam -O bam -o $1.sort.bam
sambamba markdup --remove-duplicates $1.sort.bam $1.sort.rmdup.bam ##去除pcr重复
samtools view -@ 80 -q 30 -O bam -o $1.sort.rmdup.q30.bam $1.sort.rmdup.bam  ##mapping quality按30过滤
samtools index -@ 80 $1.sort.rmdup.q30.bam
alignmentSieve --numberOfProcessors 80 --ATACshift -b $1.sort.rmdup.q30.bam -o $1.sort.rmdup.q30.shift.bam ##移码
bedtools bamtobed -i $1.sort.rmdup.q30.shift.bam > $1.sort.rmdup.q30.shift.bed
samtools sort $1.sort.rmdup.q30.shift.bam -@ 80 -O bam -o $1.sort.rmdup.q30.shift.sort.bam
samtools index -@ 80 $1.sort.rmdup.q30.shift.sort.bam
bamCoverage -o $1.sort.rmdup.q30.shift.sort.bw -b $1.sort.rmdup.q30.shift.sort.bam -p 40 --normalizeUsing RPGC --effectiveGenomeSize 2100000000 --binSize 10
macs2 callpeak --format BED --gsize 2.1e+9 --keep-dup all --qvalue 0.05 --extsize 200 --shift -100 --nomodel -t $1.sort.rmdup.q30.shift.bed -n $1 --outdir ./


##usage example sbatch ./peakcalling.sh tzi8

### 对ocr与基因的关系进行注释，

### filter OCRs by Tn5 integration site density

In [ ]:
##提取reads的Tn5 site
awk 'BEGIN {OFS = "\t"} ; {if ($6 == "+") print $1,$2,$2+1;else print $1,$3-1,$3}' b73.sort.rmdup.q30.shift.bed > b73.sort.rmdup.q30.shift.Tn5.bed
##计算每个ocr的tn5 site densify（tn5 count/ ocr length)
bedtools coverage -a b73_peaks.q005.narrowPeak -b b73.sort.rmdup.q30.shift.Tn5.bed | awk '{print $1,$2,$3,$4,$11}' > input_b73_peaks_and_tn5count.bed
##bedtools shuffle specifically excluding ACRs from the randomized selection space
bedtools shuffle -i b73_peaks.q005.narrowPeak -g ./Zm-B73-REFERENCE-NAM-5.0.fa.wo_scaf.fai |awk '{print $1,$2,$3}' | tr ' ' '\t' > shuffle_randomized.bed ##fai先把scaff去掉
##计算randomized区域的tn5 density
bedtools coverage -a shuffle_randomized.bed -b b73.sort.rmdup.q30.shift.Tn5.bed | awk '{print $1,$2,$3,$4}' > input_randomized_and_tn5count.bed #四列 chr start end tn5count

Rscript ./filter_eFDR.R input_b73_peaks_and_tn5count.bed input_randomized_and_tn5count.bed 0.01 b73_peaks_filted_by_tn5density.fdr001.bed


#########-----------------filter_ocr_by_tn5_density.sh-------#############
#!/bin/bash
sample_name=$1
PATH_to_sort.rmdup.shift.bed=$2
PATH_to_initial_narrowpeak=$3
PATH_to_genome_fai=$4
fdr=$5
##提取reads的Tn5 site
awk 'BEGIN {OFS = "\t"} ; {if ($6 == "+") print $1,$2,$2+1;else print $1,$3-1,$3}' $2 > $2.Tn5
##计算每个ocr的tn5 site densify（tn5 count/ ocr length)
bedtools coverage -a $3 -b $2.Tn5 | awk '{print $1,$2,$3,$4,$11}' > input_$1_peaks_and_tn5count.bed
##bedtools shuffle specifically excluding ACRs from the randomized selection space
grep -v 'scaf' $4 > $4.woscaf
bedtools shuffle -i $3 -g $4.woscaf |awk '{print $1,$2,$3}' | tr ' ' '\t' > $1.shuffle_randomized.bed ##fai先把scaff去掉
##计算randomized区域的tn5 density
bedtools coverage -a $1.shuffle_randomized.bed -b $2.Tn5 | awk '{print $1,$2,$3,$4}' > input_$1_randomized_and_tn5count.bed #四列 chr start end tn5count

Rscript ./filter_eFDR.R input_$1_peaks_and_tn5count.bed input_$1_randomized_and_tn5count.bed $5 $1_peaks_filted_by_tn5density.fdr$5.bed
##usage example: ./filter_ocr_by_tn5_density.sh b73 ./b73.sort.rmdup.q30.shift.bed ./b73_peaks.q005.narrowPeak ./Zm-B73-REFERENCE-NAM-5.0.fa.fai 0.01
##会有个报错，但是不影响使用


#########-----------------filter_ocr_by_tn5_density_fit_zkd.sh-------#############
#!/bin/bash
sample_name=$1
PATH_to_sort.rmdup.shift.bed=$2
PATH_to_initial_narrowpeak=$3
PATH_to_genome_fai=$4
fdr=$5
##提取reads的Tn5 site
awk 'BEGIN {OFS = "\t"} ; {print $1,($2+$3)/2,($2+$3)/2+1}' $2 > $2.Tn5
##计算每个ocr的tn5 site densify（tn5 count/ ocr length)
bedtools coverage -a $3 -b $2.Tn5 | awk '{print $1,$2,$3,$4,$11}' > input_$1_peaks_and_tn5count.bed
##bedtools shuffle specifically excluding ACRs from the randomized selection space
bedtools shuffle -i $3 -g $4 | awk '{print $1,$2,$3}' | tr ' ' '\t' > $1.shuffle_randomized.bed ##fai先把scaff去掉
##计算randomized区域的tn5 density
bedtools coverage -a $1.shuffle_randomized.bed -b $2.Tn5 | awk '{print $1,$2,$3,$4}' > input_$1_randomized_and_tn5count.bed #四列 chr start end tn5count

Rscript ./filter_eFDR.R input_$1_peaks_and_tn5count.bed input_$1_randomized_and_tn5count.bed $5 $1_peaks_filted_by_tn5density.fdr$5.bed
##usage example: ./filter_ocr_by_tn5_density_zkd.sh b73 ./b73.pe.q10.sort.rmdup.shift.bed ./b73_peaks.narrowPeak ./Zm-B73-REFERENCE-NAM-5.0.fa.fai.woscaf 0.01
##会有个报错no such file or directory，但是不影响使用


In [ ]:
##tn5 过滤后的peak文件最后一列是tn5 site number, 回帖到原narrowpeak文件，提取其他列信息
awk 'NR==FNR{a[$4]=$0;next}NR>FNR{if($4 in a)print a[$1],a[$2],a[$3],a[$4],$6,$7,$8,$9,$10}' b97_peaks.narrowPeak b97_peaks_filted_by_tn5density.fdr0.01.bed | awk -F ' ' '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10}' | tr ' ' '\t' > b97_peaks_final.bed


